# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [9]:
# Your code here; import the necessary packages
import sqlite3

In [10]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')
cursor = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [11]:
# Your code here
query = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderDate = '2003-01-31'
);
"""

# Execute the query
cursor.execute(query)
results = cursor.fetchall()
print(results)

[(141, 'Freyre', 'Diego ')]


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [12]:
# Your code here
query = """
SELECT
    productName,
    SUM(quantityOrdered) AS totalItemsSold
FROM products
JOIN orderDetails USING (productCode)
GROUP BY productName
ORDER BY totalItemsSold DESC;
"""

# Execute the query
cursor.execute(query)
results = cursor.fetchall()
print(results)


[('1992 Ferrari 360 Spider red', 1808), ('1937 Lincoln Berline', 1111), ('American Airlines: MD-11S', 1085), ('1941 Chevrolet Special Deluxe Cabriolet', 1076), ('1930 Buick Marquette Phaeton', 1074), ('1940s Ford truck', 1061), ('1969 Harley Davidson Ultimate Chopper', 1057), ('1957 Chevy Pickup', 1056), ('1964 Mercedes Tour Bus', 1053), ('1956 Porsche 356A Coupe', 1052), ('Corsair F4U ( Bird Cage)', 1051), ('F/A 18 Hornet 1/72', 1047), ('1980s Black Hawk Helicopter', 1040), ('1913 Ford Model T Speedster', 1038), ('1997 BMW R 1100 S', 1033), ('1972 Alfa Romeo GTA', 1030), ('1962 Volkswagen Microbus', 1029), ('2002 Suzuki XREO', 1028), ('The USS Constitution Ship', 1020), ('2001 Ferrari Enzo', 1019), ('1960 BSA Gold Star DBD34', 1015), ('1997 BMW F650 ST', 1014), ('1957 Corvette Convertible', 1013), ('18th century schooner', 1011), ('1900s Vintage Tri-Plane', 1009), ('1996 Moto Guzzi 1100i', 999), ('1940 Ford Pickup Truck', 999), ('1926 Ford Fire Engine', 998), ('1982 Camaro Z28', 997),

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [13]:
# Your code here
query = """
SELECT
    productName,
    COUNT(DISTINCT customerNumber) AS numCustomers
FROM products
JOIN orderDetails USING (productCode)
JOIN orders USING (orderNumber)
GROUP BY productName
ORDER BY numCustomers DESC;
"""

# Execute the query
cursor.execute(query)
results = cursor.fetchall()
print(results)

# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

[('1992 Ferrari 360 Spider red', 40), ('Boeing X-32A JSF', 27), ('1972 Alfa Romeo GTA', 27), ('1952 Alpine Renault 1300', 27), ('1934 Ford V8 Coupe', 27), ('HMS Bounty', 26), ('Corsair F4U ( Bird Cage)', 26), ('American Airlines: MD-11S', 26), ('1999 Yamaha Speed Boat', 26), ('1996 Moto Guzzi 1100i', 26), ('1995 Honda Civic', 26), ('1980s Black Hawk Helicopter', 26), ('1969 Harley Davidson Ultimate Chopper', 26), ('1941 Chevrolet Special Deluxe Cabriolet', 26), ('1940 Ford Delivery Sedan', 26), ('1937 Horch 930V Limousine', 26), ('1913 Ford Model T Speedster', 26), ('1900s Vintage Bi-Plane', 26), ('18th Century Vintage Horse Carriage', 26), ('America West Airlines B757-200', 25), ('ATA: B757-300', 25), ('2003 Harley-Davidson Eagle Drag Bike', 25), ('2002 Suzuki XREO', 25), ('1997 BMW R 1100 S', 25), ('1993 Mazda RX-7', 25), ('1962 LanciaA Delta 16V', 25), ('1960 BSA Gold Star DBD34', 25), ('1948 Porsche 356-A Roadster', 25), ('1940s Ford truck', 25), ('1940 Ford Pickup Truck', 25), ('1

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [16]:
# Your code here
query = """
SELECT DISTINCT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    o.city AS officeCity,
    o.officeCode
FROM employees e
JOIN offices o ON e.officeCode = o.officeCode
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders ord ON c.customerNumber = ord.customerNumber
JOIN orderDetails od ON ord.orderNumber = od.orderNumber
WHERE od.productCode IN (
    SELECT productCode
    FROM orderDetails
    JOIN orders ON orderDetails.orderNumber = orders.orderNumber
    GROUP BY productCode
    HAVING COUNT(DISTINCT orders.customerNumber) < 20
);
"""

# Execute the query
cursor.execute(query)
results = cursor.fetchall()
print(results)


[(1370, 'Gerard', 'Hernandez', 'Paris', 4), (1501, 'Larry', 'Bott', 'London', 7), (1337, 'Loui', 'Bondur', 'Paris', 4), (1166, 'Leslie', 'Thompson', 'San Francisco', 1), (1286, 'Foon Yue', 'Tseng', 'NYC', 3), (1612, 'Peter', 'Marsh', 'Sydney', 6), (1611, 'Andy', 'Fixter', 'Sydney', 6), (1401, 'Pamela', 'Castillo', 'Paris', 4), (1621, 'Mami', 'Nishi', 'Tokyo', 5), (1323, 'George', 'Vanauf', 'NYC', 3), (1165, 'Leslie', 'Jennings', 'San Francisco', 1), (1702, 'Martin', 'Gerard', 'Paris', 4), (1216, 'Steve', 'Patterson', 'Boston', 2), (1188, 'Julie', 'Firrelli', 'Boston', 2), (1504, 'Barry', 'Jones', 'London', 7)]


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [15]:
# Your code here
query = """
SELECT
    e.employeeNumber,
    e.firstName,
    e.lastName,
    AVG(c.creditLimit) AS avgCreditLimit
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY e.employeeNumber
HAVING AVG(c.creditLimit) > 15000;
"""

# Execute the query
cursor.execute(query)
results = cursor.fetchall()
print(results)


[(1165, 'Leslie', 'Jennings', 100433.33333333333), (1166, 'Leslie', 'Thompson', 65266.666666666664), (1188, 'Julie', 'Firrelli', 73916.66666666667), (1216, 'Steve', 'Patterson', 81533.33333333333), (1286, 'Foon Yue', 'Tseng', 66614.28571428571), (1323, 'George', 'Vanauf', 80887.5), (1337, 'Loui', 'Bondur', 86233.33333333333), (1370, 'Gerard', 'Hernandez', 91785.71428571429), (1401, 'Pamela', 'Castillo', 81340.0), (1501, 'Larry', 'Bott', 91187.5), (1504, 'Barry', 'Jones', 85511.11111111111), (1611, 'Andy', 'Fixter', 86060.0), (1612, 'Peter', 'Marsh', 92080.0), (1621, 'Mami', 'Nishi', 83900.0), (1702, 'Martin', 'Gerard', 71916.66666666667)]


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!